In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! cp -r /content/drive/My\ Drive/Hackabit/new /content/

In [4]:
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Nadam
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [5]:
cnn_base = Xception(input_shape=(240,240,3),weights="imagenet",include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(input=cnn_base.input, output=cnn_out)
cnn.trainable = False
cnn.summary()













83689472/83683744 [==============================] - 7s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 119, 119, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 119, 119, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 119, 119, 32) 0           block1_conv1_bn[0][0]            
_______________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
encoded_frames = Input((10, 2048))
encoded_sequence = LSTM(256)(encoded_frames)
hidden_layer = Dense(output_dim=1024, activation="relu")(encoded_sequence)
outputs = Dense(output_dim=2, activation="softmax")(hidden_layer)
model = Model([encoded_frames], outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10, 2048)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               2360320   
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 2,625,538
Trainable params: 2,625,538
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1024)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=2)`
  after removing the cwd from sys.path.


In [7]:
optimizer = Nadam(lr=0.002,beta_1=0.9,beta_2=0.999,epsilon=1e-08,schedule_decay=0.004)
model.compile(loss="categorical_crossentropy",optimizer=optimizer,metrics=["categorical_accuracy"])

In [0]:
import os
import keras.preprocessing.image as kimg

def image_generator(path, labels, frames=10, batch_size=32):
    l = len(os.listdir(path))
    i = 1
    batch_x = np.ndarray((batch_size, frames, 240, 240, 3))
    batch_y = np.ndarray((batch_size, 2))
    while True:
        for j in range(batch_size):
            for k in range(frames):
                batch_x[j,k] = kimg.img_to_array(kimg.load_img(os.path.join(path, 'p'+str(i+k)+'.png'), target_size=(240, 240)))/255
                batch_y[j] = labels[i+k-1]
            i += 1

        if i > l-frames-batch_size:
            i = 1
        yield batch_x, batch_y

In [0]:
t = [[26781,30386],
     [77509,79892],
     [81790,87836],
     [131366,132776],
     [135979,157150],
     [161739,163247],
     [166900,168463],
     [177235,178336],
     [184014,185293],
     [188466,188504],
     [198250,213895]]
l = 9260
labels = np.ndarray((l, 2))
for i in range(l):
    chngd = False
    for j in t:
        if i+1>=round(j[0]/23.98) and i+1<=round(j[1]/23.98):
            labels[i] = [0,1]
            chngd = True
            break
        if not chngd:
            labels[i] = [1,0]
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [0]:
train_data = kimg.ImageDataGenerator(rescale=1/255).flow_from_directory('/content/imgs',target_size = (240,240))
predicted = cnn.predict

In [0]:
# checkpointer = callbacks.ModelCheckpoint(filepath='/content/drive/My\ Drive/weight/inception_weights.hdf5',verbose = 1,save_best_only=True)
hist = model.fit_generator(image_generator('/content/imgs', labels), epochs=5, steps_per_epoch=l//32)
plt.plot(hist.history['loss'])

Epoch 1/5


ResourceExhaustedError: ignored

In [0]:
l//32

289